In [1]:
import os, glob, re
import pandas as pd
import numpy as np
from collections import defaultdict
num = 3

In [2]:
path = "exported/2." + str(num)
filenames = glob.glob(path + "/*.xlsx")
missed_years = {
    'exported/2.' + str(num) + '/table-2.' + str(num)+'.xlsx': '2015',
    'exported/2.' + str(num) + '/table-2.' +str(num)+'_0.xlsx': '2016',
    'exported/2.' +str(num)+ '/table-2.' +str(num) +'_1.xlsx': '2017'
}

In [3]:
path

'exported/2.3'

In [4]:
def get_year(name):
    pat = re.findall('\d{4}', name)
    if len(pat) != 0:
        return pat[0]
    else:
        return missed_years[name]

In [5]:
filenames = [(f, get_year(f)) for f in filenames]

In [6]:
filenames

[('exported/2.3/table-2.3_2012_0.xlsx', '2012'),
 ('exported/2.3/atable 2.3_2003.xlsx', '2003'),
 ('exported/2.3/table-2.3_2013.xlsx', '2013'),
 ('exported/2.3/table-2.3_2009.xlsx', '2009'),
 ('exported/2.3/table-2.3_2012.xlsx', '2012'),
 ('exported/2.3/Table 2.3_2005.xlsx', '2005'),
 ('exported/2.3/table-2.3_2008_0.xlsx', '2008'),
 ('exported/2.3/table-2.3_2011.xlsx', '2011'),
 ('exported/2.3/Table 2.3_2004.xlsx', '2004'),
 ('exported/2.3/table-2.3_2010.xlsx', '2010'),
 ('exported/2.3/TABLE - 2.3_2001.xlsx', '2001'),
 ('exported/2.3/table-2.3_0.xlsx', '2016'),
 ('exported/2.3/table-2.3_1.xlsx', '2017'),
 ('exported/2.3/Table-2.3_2007.xlsx', '2007'),
 ('exported/2.3/table-2.3_2008.xlsx', '2008'),
 ('exported/2.3/Table-2.3_2006.xlsx', '2006'),
 ('exported/2.3/table-2.3_2014.xlsx', '2014'),
 ('exported/2.3/atable 2.3_2002.xlsx', '2002'),
 ('exported/2.3/table-2.3.xlsx', '2015')]

In [7]:
dfs = []
for filename, year in filenames:
    dfs.append(pd.read_excel(filename, sheet_name=None, index_col=0))
    
def matchv2(s):
    if type(s) is str:
        match = re.findall(r"\((\d+)\)", s)
        return match if len(match) != 0 else None
    else:
        return None
    
def set_columns(df, index):
    frame = df.copy()
    frame.columns = frame.iloc[index].apply(lambda x : x.strip() if isinstance(x, str) else x )
    return frame.iloc[index + 1:]

def merge_columns12(df_, idx):
    tmp =  df_[df_.columns[idx]].str.extract('(\d+)\s+(\w.*)')
    df_ = df_.drop(['(1) \n(2)'], axis=1)
    return df_.join(tmp.rename(columns={0:'(1)', 1:'(2)'}))
   

In [8]:
def merge_columns(df_, idx):
    global df_g, idx_g
    df_g = df_
    idx_g = idx
    tmp =  df_[df_.columns[idx]].str.extract('(\d+)\s+(\w.*)')
    if '(1) \n(2)' in df_.columns:
        df_ = df_.drop(['(1) \n(2)'], axis=1)
        return df_.join(tmp.rename(columns={0:'(1)', 1:'(2)'}))
    else:
        raise Exception("multiple columns")

In [9]:
def safeisnan(s):
    if type(s) is str:
        return False
    else:
        return np.isnan(s)
    
    
def return_df(f):
    cols, acc = f.columns, {}
    for c in range(len(cols)-1):
        if not (safeisnan(cols[c]) or safeisnan(cols[c+1])):
            acc[cols[c]] = f[cols[c]]
        elif safeisnan(cols[c]) and not safeisnan(cols[c+1]):
             pass
        elif not safeisnan(cols[c]) and safeisnan(cols[c+1]):
            acc[cols[c]] = f.iloc[:, c+1]
        else:
          #  raise Exception("unanticipated")
            pass
    else:
        acc[cols[c+1]] = f.iloc[:, c+1]
    return pd.DataFrame(acc)

In [10]:
def rotate_right(cols, n):
    print(cols, n)
    return cols[-n:]+cols[:-n]

In [11]:
data = defaultdict(dict)
suspects = defaultdict(dict)

for excel, (filename, year) in zip(dfs, filenames):
    for subsheet in excel.keys():
        arr = excel[subsheet].applymap(matchv2).values
        idx = [j for j in range(len(arr)) 
               if any([val is not None for val in arr[j]])
              ]
        if len(idx) != 0:
            df = set_columns(excel[subsheet], idx[0])
            lst = [i for i, val in enumerate(arr[idx][0]) if isinstance(val, list) and len(val) > 1]
            if len(lst) > 0:
                df = merge_columns(df, lst[0])
            else:
                df['(1) \n(2)'] = df['(1)'] + " " + df['(2)'] 
                df = df.drop(['(1)', '(2)'], axis=1)
                df = merge_columns(df, df.shape[1] - 1)
                
            if len(df.columns[df.columns.isna()]) != 0:
                df = return_df(df)  
            columns = df.columns
            print(df)
        else:
            
            df = excel[subsheet].copy()
            print(df)
            df = df.loc[:, df.isna().mean() <= 0.5]
#             df['year'] = year
            df.columns = rotate_right(columns.to_list(), 2)

            
        df['year'] = year
        data[filename][subsheet] = df.set_index('(2)').dropna()


    (3)     (4)    (5)  (1)                (2)
4   NaN     NaN    NaN  NaN                NaN
5   NaN   14238   -5.6    1     ANDHRA PRADESH
6   NaN     130   -3.0    2  ARUNACHAL PRADESH
7   NaN    3264   19.7    3              ASSAM
8   NaN     759   -4.5    4              BIHAR
9   NaN    5654  -16.3    5       CHHATTISGARH
10  NaN     289   -1.4    6                GOA
11  NaN    7110   11.4    7            GUJARAT
12  NaN    2827  -12.9    8            HARYANA
13  NaN     528   19.2    9   HIMACHAL PRADESH
14  NaN     414   44.3   10    JAMMU & KASHMIR
15  NaN    1319    8.8   11          JHARKHAND
16  NaN   12753    1.0   12          KARNATAKA
17  NaN    8490    0.7   13             KERALA
18  NaN    9775    5.6  NaN                NaN
19  NaN   16112    1.0  NaN                NaN
20  NaN      41   24.2  NaN                NaN
21  NaN     128  -16.3  NaN                NaN
22  NaN     173   92.2  NaN                NaN
23  NaN      30   -9.1  NaN                NaN
24  NaN    50

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
filename

In [ ]:
rotate_right(columns.to_list(), 2)

In [ ]:
df.columns

In [ ]:
f = (pd.concat([f for d in data.values() for f in d.values()])
     .reset_index()
     .set_index(['(2)', 'year'])
     .sort_index()
     .filter(regex=r'\(\d+\)')
     .drop('(1)', axis=1)
    )

In [ ]:
#f.to_csv('/home/kakade/k.csv')

In [ ]:
output = (f.groupby(f.index)
          .apply(lambda df : df.fillna(method='ffill'))
          .reset_index()
          .drop_duplicates(subset=['(2)', 'year'], keep='last')
          .set_index(['(2)', 'year'])
    ).to_csv('/home/kakade/2.'+ str(num)+'.csv')